# Allstate Claims Severity - Initial Exploration / データ分析 (EDA)

## 1. Import Libraries / ライブラリのインポート
Import necessary libraries (Pandas, NumPy, Matplotlib, Seaborn).
必要なライブラリ（Pandas, NumPy, Matplotlib, Seaborn）を読み込みます。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
pd.set_option('display.max_columns', None)

## 2. Load Data / データの読み込み
Load the CSV files.
CSVファイルを読み込みます。

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sample_submission = pd.read_csv('../input/sample_submission.csv')

print(f'Train shape (学習データ): {train.shape}')
print(f'Test shape (テストデータ): {test.shape}')

## 3. Basic Inspection / 基本的なデータの確認
Check the first few rows and data types (info).
データの先頭行や、型情報(info)を確認します。

In [ ]:
train.head()

In [ ]:
train.info()

## 4. Target Variable Analysis (loss) / 目的変数 (loss) の分析
Analyze the distribution of the target variable `loss`.
予測対象である `loss`（損害額）がどのような分布になっているか確認します。

If the distribution is skewed, log transformation is often applied.
偏りが大きい場合、対数変換（log）をして分布を整えることが一般的です。

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(train['loss'], kde=True, bins=50)
plt.title('Distribution of Loss / 損害額の分布')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(np.log1p(train['loss']), kde=True, bins=50)
plt.title('Distribution of Log(Loss) / 対数変換後の分布')
plt.show()

## 5. Correlation Analysis / 相関分析
Check correlation between continuous variables (`cont1` ~ `cont14`) and `loss`.
連続値の変数 (`cont1` ~ `cont14`) と `loss` の相関を確認します。

High correlation suggests features share similar characteristics.
相関係数が高い変数は、互いに似た性質を持っている可能性があります。

In [ ]:
# List columns containing 'cont' / 列名に 'cont' を含むものをリストアップ
cont_features = [col for col in train.columns if 'cont' in col]
corr = train[cont_features + ['loss']].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Matrix (Continuous Features + Loss)\n相関行列（連続値変数 ＋ 損害額）')
plt.show()

## 6. Scatter Plots: Continuous Features vs Loss / 散布図
Visualize the relationship between each continuous feature and `loss`.
各連続値変数と `loss` の関係を散布図で可視化します。

Sampling 1000 points for better visibility.
データ数が多いので、1000件だけランダムに抽出して傾向を見やすくします。

In [ ]:
fig, axes = plt.subplots(nrows=7, ncols=2, figsize=(15, 30))

for i, col in enumerate(cont_features):
    row = i // 2
    col_idx = i % 2
    sns.scatterplot(data=train.sample(1000), x=col, y='loss', ax=axes[row, col_idx], alpha=0.5)
    axes[row, col_idx].set_title(f'{col} vs Loss')

plt.tight_layout()
plt.show()